### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

* While female players make up a smaller percentage of total players, female player spend more on average ($\$$4.47 per female player) as compared to their male counterparts ($\$$4.07 per male player).

* Spending by age group does not follow a clear pattern; the highest average purchase rates occur in the 35-39 age cohort followed by the under 10 cohort. Players over 40 represented the smallest demographic and also had the lowest average purchase price.
* The top selling items by purchase count and by total purchase value tended toward the higher end of the price scale (closer to the $\$$4.99 maximum price than to the $\$$1.00 minimum), likely corresponding to the higher desirability of items with more power and thus higher cost. The exception was the Pursuit, Cudgel of Necromancy, which had a high purchase rate (8) along with a low cost ($\$$1.02), suggesting that players find this item to be a good value for game play.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Player Count

In [2]:
#Create a data frame containing unique players
players = pd.DataFrame(purchase_data["SN"].unique())
#Count unique players and return result in new data frame
player_count = pd.DataFrame(players.count(), columns=["Total Players"])

In [3]:
###Display Number of players
player_count

,Total Players
0,576


# Purchasing Analysis (Total)

In [4]:
#Calculate the number of unique items
num_of_unique_items = purchase_data["Item ID"].nunique()

In [5]:
#Calculate the average item price
avg_price = float(purchase_data["Price"].mean())

In [6]:
#Calculate the total number of purchases
number_of_purchases = purchase_data["Purchase ID"].count()

In [7]:
#Calculate total revenue
total_revenue = float(purchase_data["Price"].sum())

In [8]:
#Create a dataframe to hold results
basic_calcs = pd.DataFrame([[num_of_unique_items, '${:,.2f}'.format(avg_price), 
               number_of_purchases, '${:,.2f}'.format(total_revenue)]], 
                           columns=["Number of Unique Items", "Average Price", 
                                    "Number of Purchases", "Total Revenue"])


In [9]:
###Display the summary data frame
basic_calcs

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

In [10]:
#Create a data frame containing the gender counts for the unique players
gender_df = pd.DataFrame(purchase_data.groupby(["Gender"])["SN"].nunique())
#Resort on highest to lowest total count
gender_df = gender_df.sort_values("SN", ascending = False)
#Rename SN column to Total Count
gender_df = gender_df.rename(columns={"SN": "Total Count"})

In [11]:
#Reset index for later data frame merging
gender_df = gender_df.reset_index()

In [12]:
#Get clean player count
num_plays = purchase_data["SN"].nunique()
#Set list variable for gender percent results
gender_percent = []
#Iterate through gender to calculate and store percents
for sex in gender_df["Total Count"]:
    gender_percent.append(round((sex/num_plays)*100,2))

In [13]:
#Create data frame with gender percent results
genper_df = pd.DataFrame(gender_percent)
#Reset index for merging
genper_df.reset_index()
#Name column
genper_df.columns = ["Percentage of Players"]

In [14]:
#Merge datata frames and reset index to Gender
new_gender_df = pd.merge(gender_df, genper_df, left_index=True, right_index=True)
new_gender_df = new_gender_df.set_index(["Gender"])


In [15]:
###Display Gender Demographics
new_gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)

In [16]:
#Get counts by Gender
purchase_count = purchase_data.groupby("Gender").count()

In [17]:
#Get means by Gender
price_mean = purchase_data.groupby("Gender").mean()

In [18]:
#Get sums by Gender
price_sum = purchase_data.groupby("Gender").sum()

In [19]:
#Create new data frame with count, mean, and sum data
gender_analysis_df = pd.DataFrame({"Purchase Count" : purchase_count["Purchase ID"], 
              "Average Purchase Price" : price_mean["Price"], 
              "Total Purchase Value" : price_sum["Price"]})


In [20]:
#Calculate and add column for Avg Total Purchase per Person
gender_analysis_df["Avg Total Purchase per Person"] = gender_analysis_df["Total Purchase Value"]/new_gender_df["Total Count"]

In [21]:
###Display Gender Purchasing Analysis
gender_analysis_df.style.format({"Average Purchase Price":'${:,.2f}', 
                                 "Total Purchase Value":'${:,.2f}',
                               "Avg Total Purchase per Person": '${:,.2f}'})


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [22]:
#Create bins and labels
bins = [0, 9, 14, 19, 24, 29, 34, 39, np.inf]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", ">40"]

In [23]:
#Add Age Range column to purchase data df
purchase_data["Age Range"] = pd.cut(purchase_data["Age"], bins, labels = labels)

In [24]:
#Create new data frame containing only player age data
player_ages_df = pd.DataFrame({"Player" : purchase_data["SN"], 
                               "Age" : purchase_data["Age"], 
                               "Age Range" : purchase_data["Age Range"]})

In [25]:
#Drop duplicate players
player_ages_df.drop_duplicates("Player", inplace = True)

In [26]:
#Create new data frame with counts per age range
age_demo_df = pd.DataFrame(player_ages_df.groupby("Age Range").count())
age_demo_df = pd.DataFrame(age_demo_df["Age"])
#Change column name to Total Count
age_demo_df = age_demo_df.rename(columns = {"Age" : "Total Count"})
#Calculate and format percentage of players per age
age_demo_df["Percentage of Players"] = (age_demo_df["Total Count"]/num_plays)*100
age_demo_dfSyle = age_demo_df.style.format({"Percentage of Players":'{:,.2f}'})

In [27]:
###Display Age Demographics
age_demo_dfSyle

,Total Count,Percentage of Players
Age Range,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
>40,12,2.08


## Purchasing Analysis (Age)

In [28]:
#Calculate counts, means, and sums
byAge_purch_count = purchase_data.groupby("Age Range").count()
byAge_avgPurch_price = purchase_data.groupby("Age Range").mean()
byAge_totalPurch_value = purchase_data.groupby("Age Range").sum()

In [29]:
#Create new data frame with selected count, mean, and sum data
byAge_calcs_df = pd.DataFrame({"Purchase Count" : byAge_purch_count["Purchase ID"], 
              "Average Purchase Price" : byAge_avgPurch_price["Price"], 
              "Total Purchase Value" : byAge_totalPurch_value["Price"]})

In [30]:
#Calculate and add column for Avg Total Purchase per Person
byAge_calcs_df["Avg Total Purchase per Person"] = byAge_calcs_df["Total Purchase Value"]/age_demo_df["Total Count"]

In [31]:
###Display by age Purchasing Analysis
byAge_calcs_df.style.format({"Average Purchase Price":'${:,.2f}', 
                                 "Total Purchase Value":'${:,.2f}',
                               "Avg Total Purchase per Person": '${:,.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
>40,13,$2.94,$38.24,$3.19


## Top Spenders

In [32]:
#Calculate Purchase Count, Average Purchase Price, and Total Purchase Value for each player
playPurch_count = purchase_data.groupby("SN").count()
playPurch_avg = purchase_data.groupby("SN").mean()
playPurch_sum = purchase_data.groupby("SN").sum()

In [33]:
#Create new data frame with selected count, mean, and sum data
playPurch_calcs_df = pd.DataFrame({"Purchase Count" : playPurch_count["Purchase ID"], 
              "Average Purchase Price" : playPurch_avg["Price"], 
              "Total Purchase Value" : playPurch_sum["Price"]})
#Sort data by Total Purchase Value
playPurch_calcs_df = playPurch_calcs_df.sort_values("Total Purchase Value", ascending=False)
###Display results
playPurch_calcs_df.head().style.format({"Average Purchase Price":'${:,.2f}', 
                                 "Total Purchase Value":'${:,.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [34]:
#Calculate Purchase Count and Total Purchase Value for each item
item_count = purchase_data.groupby(["Item ID", "Item Name"]).count()
item_price = purchase_data.groupby(["Item ID", "Item Name"]).mean()
item_sum = purchase_data.groupby(["Item ID", "Item Name"]).sum()

In [35]:
#Create new data frame with selected count, mean, and sum data
item_calcs_df = pd.DataFrame({"Purchase Count" : item_count["Purchase ID"], 
              "Item Price" : item_price["Price"], 
              "Total Purchase Value" : item_sum["Price"]})
#Sort data by Purchase Count
item_calcs_df = item_calcs_df.sort_values("Purchase Count", ascending=False)
###Display results
item_calcs_df.head().style.format({"Item Price":'${:,.2f}', 
                                 "Total Purchase Value":'${:,.2f}'})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [36]:
#Sort data by Total Purchase Value
item_calcs_df = item_calcs_df.sort_values("Total Purchase Value", ascending=False)
###Display results
item_calcs_df.head().style.format({"Item Price":'${:,.2f}', 
                                 "Total Purchase Value":'${:,.2f}'})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
